# Data Exploration

Make sure to download and extract the `rba-dataset.csv` file from this Kaggle page: https://www.kaggle.com/datasets/dasgroup/rba-dataset

The data set is over 9gb, so we have to use dask dataframes to let it all fit in memory (unless you have a massive PC with 16gb of memory). It is a little annoying, so we might decide to work with a smaller sample of the data that is 500mb-1gb to make it more manageable. 

In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import seaborn as sns

In [2]:
CSV_FILENAME = '../data/rba-dataset.csv'

In [3]:
df = dd.read_csv(CSV_FILENAME) 

## How many successful and failed attempts are there by hour of day?

In [9]:
df['login_timestamp_dttm'] = dd.to_datetime(df['Login Timestamp'])
df['hour_of_day'] = df['login_timestamp_dttm'].dt.hour
df['count'] = 1
df_logins = df.groupby(['hour_of_day', 'Login Successful'])['count'].count()
df_logins.head()

hour_of_day  Login Successful
0            False               335181
             True                106112
1            False               412196
             True                123122
2            False               511461
Name: count, dtype: int64

## Successful V.S. Failed Logins

During the night time hours, there are more failures proportional to the succesful ones. 

In [ ]:
sns.barplot(
    data=df_logins.reset_index(), 
    x="hour_of_day", 
    y="count",
    hue='Login Successful'
)

## Any class imbalances? 

In [ ]:
df['count'] = 1
df.groupby(['Is Account Takeover'])['count'].sum()

## Can we predict the attach IP vector? 

Start as simple as possible then try to incorporate new features

In [11]:
df_sample = df.sample(frac=0.05, random_state=42)
df_sample.head() 

,index,Login Timestamp,User ID,Round-Trip Time [ms],IP Address,Country,Region,City,ASN,User Agent String,Browser Name and Version,OS Name and Version,Device Type,Login Successful,Is Attack IP,Is Account Takeover,login_timestamp_dttm,hour_of_day,count
15884,15884,2020-02-03 15:51:54.190,-4324475583306591935,NaN,38.135.39.46,US,-,-,393398,Mozilla/5.0 (iPhone; CPU iPhone OS 14_2_1 lik...,Chrome Mobile 81.0.4044.1949,iOS 14.2.1,mobile,False,True,False,2020-02-03 15:51:54.190,15,1
91106,91106,2020-02-04 15:25:25.352,-5043908894228897431,NaN,158.149.95.19,NO,-,-,29492,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6...,Chrome 69.0.3497.17.19,Mac OS X 10.14.6,desktop,True,False,False,2020-02-04 15:25:25.352,15,1
121880,121880,2020-02-05 00:35:47.521,-621088459309903219,NaN,91.186.5.50,GB,-,-,29550,Mozilla/5.0 (X11; CrOS armv7l 5978.98.0) Appl...,Chrome 69.0.3497.17.19,Chrome OS 5978.98.0,desktop,True,False,False,2020-02-05 00:35:47.521,0,1
36016,36016,2020-02-03 20:17:14.639,-4324475583306591935,NaN,92.221.195.76,NO,Vestland,Bergen,29695,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6...,Edge 81.0.416,Mac OS X 10.14.6,desktop,False,False,False,2020-02-03 20:17:14.639,20,1
134957,134957,2020-02-05 07:45:54.544,-4324475583306591935,NaN,23.137.224.227,US,-,-,393398,Mozilla/5.0 (iPhone; CPU iPhone OS 14_2_1 lik...,Chrome Mobile WebView 85.0.4183,iOS 14.2.1,mobile,False,False,False,2020-02-05 07:45:54.544,7,1


In [12]:
df_sample['count'] = 1
df_login_times_by_ip = df_sample.groupby(['IP Address', 'hour_of_day'])['count'].sum().reset_index()

# Note that hour is UTC normalized
df_login_times_by_ip['hour_of_day'] = df_login_times_by_ip['hour_of_day'].astype('category').cat.as_known()
df_login_times_by_ip.head() 

,IP Address,hour_of_day,count
0,10.0.0.107,20,1
1,10.0.0.113,17,2
2,10.0.0.121,15,8
3,10.0.0.134,6,6
4,10.0.0.134,13,8


In [22]:
df_pivot = df_login_times_by_ip.pivot_table(
    index='IP Address', 
    columns='hour_of_day', 
    values='count', 
    aggfunc='mean'
).add_prefix('hour_')
df_pivot = df_pivot.fillna(0)
df_pivot.head() 

hour_of_day,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
IP Address,,,,,,,,,,,,,,,,,,,,,
1.37.0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1.37.0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10.0.0.1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,2.0,...,2.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0
10.0.0.10,0.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,...,2.0,3.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0
10.0.0.100,0.0,1.0,1.0,2.0,1.0,2.0,6.0,2.0,4.0,5.0,...,4.0,3.0,2.0,0.0,2.0,1.0,2.0,1.0,0.0,0.0


In [17]:
df_success_rate_by_ip = df_sample.groupby(['IP Address'])['Login Successful'].mean().reset_index()
df_success_rate_by_ip.head() 

,IP Address,Login Successful
0,10.0.0.107,0.287500
1,10.0.0.113,0.194805
2,10.0.0.121,0.166667
3,10.0.0.134,0.219512
4,10.0.0.139,0.136364


In [18]:
df_device_breakout = df_sample.groupby(['IP Address', 'Device Type'])['count'].count().reset_index() 
df_device_breakout['Device Type'] = df_device_breakout['Device Type'].astype('category').cat.as_known()
df_device_breakout = df_device_breakout.pivot_table(
    index='IP Address', 
    columns='Device Type', 
    values='count', 
    aggfunc='mean'
)
df_device_breakout = df_device_breakout.fillna(0)
df_device_breakout.head() 

Device Type,bot,desktop,mobile,tablet,unknown
IP Address,,,,,
1.37.0.5,NaN,NaN,1.0,NaN,NaN
1.37.0.6,NaN,NaN,1.0,NaN,NaN
10.0.0.1,NaN,NaN,17.0,NaN,NaN
10.0.0.10,NaN,NaN,19.0,NaN,NaN
10.0.0.100,NaN,1.0,54.0,NaN,NaN


## Combine above dataframes to one feature set 

In [27]:
df_features = df_sample.groupby(['IP Address']).agg({'Is Attack IP': 'first'}).reset_index() 
df_features = df_features.merge(df_success_rate_by_ip, on=['IP Address'], how='left')
df_features = df_features.merge(df_pivot, on=['IP Address'], how='left')
df_features = df_features.merge(df_device_breakout, on=['IP Address'], how='left')
df_features.head() 

,IP Address,Is Attack IP,Login Successful,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,...,hour_19,hour_20,hour_21,hour_22,hour_23,bot,desktop,mobile,tablet,unknown
0,38.135.39.46,True,0.266667,2.0,1.0,3.0,5.0,4.0,9.0,9.0,...,1.0,3.0,1.0,1.0,1.0,NaN,1.0,118.0,1.0,NaN
1,158.149.95.19,False,0.825581,0.0,0.0,0.0,0.0,0.0,3.0,4.0,...,7.0,1.0,0.0,0.0,0.0,NaN,85.0,NaN,1.0,NaN
2,91.186.5.50,False,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,NaN,1.0,NaN,NaN,NaN
3,92.221.195.76,False,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,NaN,1.0,1.0,NaN,NaN
4,23.137.224.227,False,0.262069,3.0,2.0,3.0,3.0,6.0,6.0,6.0,...,3.0,1.0,4.0,0.0,2.0,NaN,NaN,145.0,NaN,NaN


In [28]:
df_features = df_features.compute() 
df_features.to_csv('sample-features.csv')

## Model building for the "Is Attack IP" variable

## Model analysis for the "Is Attack IP" variable

## Can we predict the "Is Account Takeover" variable? 


TBD.... this code doesn't work yet

In [ ]:
# df = df.sort_values('login_timestamp_dttm')
# df_gb = df.groupby(['User ID'])
# df_gb['Time Since Last Attempt'] = df_gb['login_timestamp_dttm'].rolling(window=2).apply(lambda x: x.iloc[-1] - x.iloc[0])
# df_final = df_gb.reset_index() 
# df_final.head() 